In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Large_B-cell_Lymphoma/GSE197977'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tumour immune contexture is a determinant of anti-CD19 CAR T-cell efficacy in large B cell lymphoma"
!Series_summary	"Axicabtagene ciloleucel (axi-cel), an anti-CD19 chimeric antigen receptor (CAR) T-cell therapy approved for treatment of relapsed/refractory large B-cell lymphoma (LBCL), has comparable efficacy across conventional LBCL markers. We analysed whether pre- and posttreatment  tumour immune contexture determines clinical outcomes for axi cel–treated patients in the ZUMA-1 pivotal study. Longitudinal evaluation of the tumour microenvironment (TME) uncovered dynamic patterns that occurred rapidly after axi-cel (within 2 weeks) in responders—pronounced enhancement of T- and myeloid cell signatures and diminution of B cell signature. Clinical response and overall survival associated with high CD8+ T-cell density (Immunoscore) and immune gene expression (Immunosign21) in TME pretreatment, which was paralleled by blood CAR T-cell levels postt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
# Based on the background information it seems likely as there is a specific focus on genetic expression
is_gene_available = True

# Data Availability
# 'Large_B-cell_Lymphoma' information is not explicitly provided, so it is not clearly available.
trait_row = None
# 'age' information is not explicitly provided in the sample characteristics data.
age_row = None
# 'gender' information is not explicitly provided in the sample characteristics data.
gender_row = None

# Conversion Functions
def convert_trait(value):
    return None  # As trait data is not available

def convert_age(value):
    return None  # As age data is not available

def convert_gender(value):
    return None  # As gender data is not available

# Save cohort information
save_cohort_info('GSE197977', './preprocessed/Large_B-cell_Lymphoma/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, we should skip substep 4.
